# Prediction check for manual cascading

## 1. Set up

In [ ]:
import os
import json
import glob
import scipy
import numpy as np

# import preprocess func from processor module
from processor import preprocess
from processor import DataSet

from keras.models import model_from_json
from keras.models import model_from_yaml
from keras.utils import np_utils
from keras.preprocessing import image

import pandas as pd
import glob

In [ ]:
# set root as ../
import sys
sys.path.append("../")

In [ ]:
# created by virtual_categories.ipynb
CATSTORE_PATH = ""

In [ ]:
BASE_MODEL_DIR = ""
BASE_NAME = ""
CODE_EXTRACTOR=""
BASE_CODE_NAME = "code_" + BASE_NAME

In [ ]:
from one_vs_all import ModelBinder
from codeextractor import DNNCodeExtractor

def ModelBinder_create(base_model_name = BASE_CODE_NAME, basedir = BASE_MODEL_DIR, extractor_path=CODE_EXTRACTOR):
    return ModelBinder(base_model_name, basedir, DNNCodeExtractor.create_from(extractor_path))

In [ ]:
from one_vs_all import ManualCascader

In [ ]:
CHECK_DATA_DIR1 = ''
CHECK_DATA_DIR2 = ''

In [ ]:
CHECK_DATA_PATHS1 = list(glob.glob(os.path.normpath("{}/*.jpg").format(CHECK_DATA_DIR1)))
CHECK_DATA_PATHS2 = list(glob.glob(os.path.normpath("{}/*.jpg").format(CHECK_DATA_DIR2)))

In [ ]:
import random, itertools

In [ ]:
# CHECK_DATA_PATHS = list(itertools.chain(random.sample(CHECK_DATA_PATHS1, 5000), CHECK_DATA_PATHS2))
CHECK_DATA_PATHS = list(itertools.chain(CHECK_DATA_PATHS1[0:5000], CHECK_DATA_PATHS2[0:5000]))

In [ ]:
len(CHECK_DATA_PATHS)

# Manual Cascading predict

In [ ]:
ens = ModelBinder_create()

In [ ]:
# 15 cat
key_thrds_pair = [
]

In [ ]:
# 19 cat
key_thrds_pair = [
]

In [ ]:
# 27 cat
key_thrds_pair = [
]

In [ ]:
manuens= ManualCascader(key_thrds_pair, ens)

In [ ]:
manuens.load_all_models()

# Normal predict version

In [ ]:
CHUNKSIZE=3000
chunks = [CHECK_DATA_PATHS[x:x + CHUNKSIZE] for x in range(0, len(CHECK_DATA_PATHS), CHUNKSIZE)]

In [ ]:
from tqdm import tqdm
import itertools

In [ ]:
res = [manuens.predict(chunk) for chunk in tqdm(chunks)]

In [ ]:
# flatten result
res = list(itertools.chain.from_iterable(res))

In [ ]:
len(res)

In [ ]:
respair = list(zip(res, CHECK_DATA_PATHS))

In [ ]:
len(respair)

In [ ]:
OTHER = manuens.otherclass

In [ ]:
resdf = pd.DataFrame(respair)

In [ ]:
resdf.columns = ["category", "filepath"]

In [ ]:
DF_PATH=""

In [ ]:
resdf.to_pickle(DF_PATH)

In [ ]:
resdf = pd.read_pickle(DF_PATH)

### df evaluation

In [ ]:
resdf.count()['category']

In [ ]:
resdf.where(resdf["category"] != OTHER).sort_values(by="category").dropna().count()['category']

In [ ]:
from category import VirtualCategories
import os
from visualize import plot_image_list

In [ ]:
vc = VirtualCategories.from_file(CATSTORE_PATH)

In [ ]:
def plot_category(classidx):
    print(vc.name(manuens.class2catname(classidx)))
    plot_image_list(resdf.where(resdf["category"] == classidx).dropna()["filepath"].values)


In [ ]:
plot_category(0)

In [ ]:
OTHER

In [ ]:
resdf.where(resdf["category"] == OTHER).dropna()

# Check for predict_arrs

In [ ]:
resdf = pd.read_pickle("")

In [ ]:
from processor import DataSet

In [ ]:
ds = DataSet()

In [ ]:
arrs = ds.files_to_dataset(CHECK_DATA_PATHS)

In [ ]:
arrdf = manuens.predict_arrs(arrs)

In [ ]:
arrdf[0:20]

In [ ]:
resdf[0:20]["category"].values == arrdf[0:20]